In [1]:
#Part 1 - Language identification with linear classification [ SGDClassifier and MultinomialNB ]


#Importing the tools
import csv
import re
import numpy as np
import pandas as pd


In [2]:
#Loading the Data
url_train_dev = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTOZ2rC82rhNsJduoyKYTsVeH6ukd7Bpxvxn_afOibn3R-eadZGXu82eCU9IRpl4CK_gefEGsYrA_oM/pub?gid=1863430984&single=true&output=tsv'
url_test = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vT-KNR9nuYatLkSbzSRgpz6Ku1n4TN4w6kKmFLkA6QJHTfQzmX0puBsLF7PAAQJQAxUpgruDd_RRgK7/pub?gid=417546901&single=true&output=tsv'

In [3]:
#Splitting Data into labels
from io import StringIO
import requests

def load_dataset(url):
    r = requests.get(url)
    data = r.content.decode('utf8')
    df = pd.read_csv(StringIO(data), sep='\t')
    df.columns = ['tweet', 'label']
    return df
    

In [4]:
df_train  = load_dataset(url_train_dev)
df_test = load_dataset(url_test)
print(df_train[0:10])


                                               tweet label
0  يا من أناديها ويخنقني البكاء  ويكاد صمت الدمع ...    ar
1  فيه فرق بين اهل غزة اللى مطحونين من ناحيتين وب...    ar
2  ﻋﻦ ﺍﻟﻠﺤﻈﺔ اﻟﺤﻠﻮﺓﺓ ﺍﻟﻠﻲ ﺑﺘﻐﻤﺾ ﻓﻴﻬﺎ ﻋﻴﻨﻴﻚ ﺑﺘﻔﻜﺮ ...    ar
3                                  يا ابو سلو عرفتني    ar
4  ب50 ريال أكفل معتمر في رمضان ، ولك بإذن الله م...    ar
5  توجيه كيفية تثبيت البرامج الثابتة ROM التحميل ...    ar
6  {وأنه هو أغنى وأقنى} [النجم:48] http://t.co/is...    ar
7  اللهم قدر لنا الفرح بكل اشكاله ، انت الكريم ال...    ar
8  #غزه_تحت_القصف  داعش أخواني حيل عندكم بالمدنيي...    ar
9  {يعلمون ظاهرا من الحياة الدنيا وهم عن الآخرة ه...    ar


In [5]:
# Data Preprocessing

df_train["tweet"] = df_train["tweet"].str.lower()
filter_dt=df_train.groupby('label').count()['tweet']


#Removing all tweets with less than 15 samples to train from 
filter_dt=filter_dt.loc[filter_dt.values>25].index
df_train=df_train[df_train['label'].isin(filter_dt)]
df_train.reset_index(inplace=True)
filter_dt=df_train.groupby('label').count()['tweet']

#Removing all labels from test which are not there in training
df_test=df_test[df_test['label'].isin(df_train.label.unique())]
df_test.reset_index(inplace=True)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52325 entries, 0 to 52324
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   52325 non-null  int64 
 1   tweet   52325 non-null  object
 2   label   52325 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [7]:
df_train.head()

,index,tweet,label
0,0,يا من أناديها ويخنقني البكاء ويكاد صمت الدمع ...,ar
1,1,فيه فرق بين اهل غزة اللى مطحونين من ناحيتين وب...,ar
2,2,ﻋﻦ ﺍﻟﻠﺤﻈﺔ اﻟﺤﻠﻮﺓﺓ ﺍﻟﻠﻲ ﺑﺘﻐﻤﺾ ﻓﻴﻬﺎ ﻋﻴﻨﻴﻚ ﺑﺘﻔﻜﺮ ...,ar
3,3,يا ابو سلو عرفتني,ar
4,4,ب50 ريال أكفل معتمر في رمضان ، ولك بإذن الله م...,ar


In [8]:
df_train.label.unique()

array(['ar', 'de', 'el', 'en', 'es', 'fr', 'he', 'id', 'it', 'ja', 'ko',
       'ms', 'nl', 'pl', 'pt', 'ru', 'sv', 'th', 'tl', 'tr', 'und'],
      dtype=object)

In [9]:
#Retrieving the labels from the training and the test set.
X_train = df_train.tweet
y_train = df_train.label
X_test = df_test.tweet
y_test = df_test.label

print('Training tweet shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test tweet shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training tweet shape:  (52325,)
Training labels shape:  (52325,)
Test tweet shape:  (13190,)
Test labels shape:  (13190,)


In [10]:
#Data Cleaning
def preprocessor(text):
  
  text = re.sub(r"http\S+", "", text)       ##Removing URL
  text = re.sub('@[^\s]+','',text)          ##Removing @Username
  text = re.sub('[\W]+', ' ', text.lower()) ##Removing NonCharacter eg.emojis
  text = re.sub('<[^>]*>', '', text)        ##Removing entire HTML
  text = re.sub(r'[0-9]+', '', text)        ## Removing numbers
  
  return text

X_train = X_train.apply(preprocessor)


In [11]:
X_train[0:50]

0     يا من أناديها ويخنقني البكاء ويكاد صمت الدمع أ...
1     فيه فرق بين اهل غزة اللى مطحونين من ناحيتين وب...
2     ﻋﻦ ﺍﻟﻠﺤﻈﺔ اﻟﺤﻠﻮﺓﺓ ﺍﻟﻠﻲ ﺑﺘﻐﻤﺾ ﻓﻴﻬﺎ ﻋﻴﻨﻴﻚ ﺑﺘﻔﻜﺮ ...
3                                     يا ابو سلو عرفتني
4     ب ريال أكفل معتمر في رمضان ولك بإذن الله مثل أ...
5     توجيه كيفية تثبيت البرامج الثابتة rom التحميل ...
6                            وأنه هو أغنى وأقنى النجم  
7     اللهم قدر لنا الفرح بكل اشكاله انت الكريم الذي...
8      غزه_تحت_القصف داعش أخواني حيل عندكم بالمدنيين...
9      يعلمون ظاهرا من الحياة الدنيا وهم عن الآخرة ه...
10               افضل كتاب قرأته هو أمي ابراهام لنكولن 
11    ولأن ه م م لائ ك ة ص غار ن ع ش ق ات كاءة رؤوس ...
12        خ لاصة الح ب هي ت فكر بقلبهآ وهو ي فكر بعقله 
13    جميل آن يفهمك من تحبب ويخآف عليك و يغآر عليكك ...
14    حتى الندم على المعصيه تؤجر عليه سبحانك يالله م...
15     اها يا بيبي والله اتهرست علي تويتر و ع الفيس ...
16     لا يقاتلونكم جميعا إلا في قرى محصنة أو من ورا...
17          طبت منك نهائيا بس كلي رجاء ما ترجع م

In [12]:
print(X_train.shape)
print(y_train.shape)


X_train = df_train.tweet
y_train = df_train.label
X_test = df_test.tweet
y_test = df_test.label


(52325,)
(52325,)


In [13]:
# from sklearn.model_selection import train_test_split             
# from keras.preprocessing.text import Tokenizer                    
# from keras.preprocessing.sequence import pad_sequences

# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X_train)

# X_X_train = tokenizer.texts_to_sequences(X_train)
# X_X_test = tokenizer.texts_to_sequences(X_test)
# # Adding 1 because of  reserved 0 index
# vocab_size = len(tokenizer.word_index) + 1                          

# maxlen = 100

# X_training = pad_sequences(X_X_train, padding='post', maxlen=maxlen)
# X_testing = pad_sequences(X_X_test, padding='post', maxlen=maxlen)

# X_training.shape


In [14]:
# from keras.models import Sequential
# from keras import layers
# embedding_dim = 100

# model = Sequential()
# model.add(layers.Embedding(vocab_size, 100, input_length=maxlen))
# model.add(layers.Conv1D(128, 5, activation='relu'))
# model.add(layers.GlobalMaxPooling1D())
# model.add(layers.Dense(10, activation='relu'))
# model.add(layers.Dense(1, activation='softmax'))
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# history = model.fit(X_training, y_train,
#                     epochs=10,
#                     validation_data=(X_testing, y_test),
#                     batch_size=10)  

# history

In [15]:
import itertools
import os

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_posts = df_train.tweet
train_tags = df_train.label
test_posts = df_test.tweet
test_tags = df_test.label

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/2
1472/1472 [==============================] - 4s 3ms/step - loss: 0.7296 - accuracy: 0.8203 - val_loss: 9.5068 - val_accuracy: 0.0023
Epoch 2/2
1472/1472 [==============================] - 4s 3ms/step - loss: 0.5191 - accuracy: 0.8736 - val_loss: 11.2793 - val_accuracy: 0.0023


In [16]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

413/413 [==============================] - 1s 2ms/step - loss: 1.6949 - accuracy: 0.7786
Test accuracy: 0.778620183467865
